In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt

In [2]:
idf = []
tf = []
tf_idf = []
bag_of_words = []

with open('/content/drive/MyDrive/Data/train-data.dat') as data:
    for line in data:
        tf_bag = [0 for _ in range(8520)]
        bag = [0 for _ in range(8520)]
        inline = line.strip().split()
        for index, value in enumerate(inline):
            if value[0] != '<':
                bag[int(value)] += 1
      
        bag_of_words.append(bag)
        
        # compute tf
        non_zero = np.count_nonzero(bag)
        for index, value in enumerate(bag):
          tf_bag[index] = value / non_zero
        tf.append(tf_bag)

np.array(bag_of_words)
N = np.size(bag_of_words,0)
M = np.size(bag_of_words,1)
sum_d_t = (np.count_nonzero(bag_of_words, axis=0))

# compute idf
for i in range(M):
    idf.append( np.log10( N / sum_d_t[i]) )

tf = np.array(tf)
idf = np.array(idf)

# cumpute tf-idf
tf_idf = tf * idf
tf_idf = np.array(tf_idf)


# tf_idf_mean
tf_idf_mean = np.mean(tf_idf, axis = 0)
max_fitness = (sum(tf_idf_mean))

In [3]:
class Individual(object):
    def __init__(self, chromosome):
        self.chromosome = chromosome
        self.fitness = self.calculate_fitness()

    @classmethod
    # creates a random chromosome
    def create_genome(self):

        genome = np.random.randint(0, 1, size=8520)
        rand_ones = random.randint(0, 8519)
        for _ in range(rand_ones):
            index = random.randint(0, 8519)
            
            while genome[index] == 1:
                index = random.randint(0, 8519)
            
            genome[index] = 1
                
        return genome


    def repair_genome(self):
        
        non_zeros = np.count_nonzero(self.chromosome)
        
        while non_zeros < 1000:  
            difference = 1000 - non_zeros
            for _ in range(difference):
                self.chromosome[random.randint(0, 8519)] = 1

            non_zeros = np.count_nonzero(self.chromosome)
        
        return self.chromosome

    
    def calculate_fitness(self):
        global max_fitness
        global tf_idf_mean
        global generation

        fitness = 0
        for index, gene in enumerate(self.chromosome):
            if gene == 1:
                fitness += tf_idf_mean[index]

        if np.count_nonzero(self.chromosome) > 3000:
            percentage = ((np.count_nonzero(self.chromosome) // 100) / 100)
            fitness -= ((percentage/2) * max_fitness)
    
        

        return fitness

In [4]:
def initialize_population(no_of_chromosomes):
    population = []
    for _ in range(no_of_chromosomes):

        genome = Individual.create_genome()
        genome = Individual(genome)
        genome.repair_genome()
        
        # population (list) consists Individual objects
        population.append(genome)   

    return np.array(population)

def tournament_selection(population: list):
    k = 5
    selected_parents = []
    for individual in range(k):
        index = random.randint(0, len(population) - 1)
        selected_parents.append(population[index])
    
    sorted_parents = sorted(selected_parents, key=lambda x: x.fitness, reverse = True)
    first_parent, second_parent = sorted_parents[0], sorted_parents[1]

    return first_parent, second_parent


def mate(papa: Individual, mama: Individual, p_cross):
    # 2-point crossover
    points = random.sample(range(len(mama.chromosome)), 2)
    points = sorted(points)

    papa_copy = papa.chromosome.copy()
    mama_copy = mama.chromosome.copy()

    if random.random() <= p_cross:
        papa.chromosome[0:points[0]] = mama_copy[0:points[0]]
        mama.chromosome[0:points[0]] = papa_copy[0:points[0]]
        papa.chromosome[points[1]:] = mama_copy[points[1]:]
        mama.chromosome[points[1]:] = papa_copy[points[1]:]

    first_child, second_child = Individual(papa.chromosome), Individual(mama.chromosome)
    first_child.fitness = first_child.calculate_fitness()
    second_child.fitness = second_child.calculate_fitness()
    
    return first_child, second_child

def mutate(child: Individual, p_mutate):
    if random.random() <= p_mutate:
        no_to_mutate = (5 * len(child.chromosome)) // 100
        for _ in range(no_to_mutate):

            random_index = random.randint(0, len(child.chromosome)-1)
            child.chromosome[random_index] = int(not child.chromosome[random_index])

    new_child = Individual(child.chromosome)
    new_child.fitness = new_child.calculate_fitness()
    return new_child


In [ ]:
def main():
    best_individuals = []
    generation = 1
    population = initialize_population(20)
    found = False


    
    while not found:
        population = sorted(population, key = lambda x: x.fitness, reverse=True)

        if population[0].fitness > 1.8:
            found = True
            break


        new_generation = []
        # elitism
        s = int((10*len(population)//100))
        new_generation.extend(population[:s])

        # mate and mutate the rest of the population
        s = int((90*len(population)//100))
        for _ in range(s//2):

            first_parent, second_parent = tournament_selection(population)
            #first_parent, second_parent = select(population),select(population)
            child1, child2 = mate(first_parent,second_parent,0.6)
            
            ch1 = mutate(child1,0.010)
            ch2 = mutate(child2,0.010)
            
            
            new_generation.append(ch1)
            new_generation.append(ch2)
        
        population = new_generation
        
        is_equal = 0
        
        best_individuals.append(population[0])
        
        if generation >= 2:
            if best_individuals[generation-2] == best_individuals[generation-1]:
                is_equal += 1
        
        # if best individual remains static over 20 generations, break
        # or if generation is 1000, break
        if is_equal == 20 or generation == 1000:
            break



        print(f"Generation: {generation}\t Fitness: {population[0].calculate_fitness()}\t # of genes {np.count_nonzero(population[0].chromosome)}")
        

        generation += 1

    fitnesses = [individual.fitness for individual in best_individuals]
    length  = [i for i in range(len(fitnesses))]
    plt.plot(length,fitnesses)
if __name__ == '__main__':
    main()
    
